# Science User Case - Inspecting a Candidate List

Ogle et al. (2016) mined the NASA/IPAC Extragalactic Database (NED) to identify a new type of galaxy: Superluminous Spiral Galaxies. Here's the paper:

Here's the paper: https://ui.adsabs.harvard.edu//#abs/2016ApJ...817..109O/abstract

Table 1 lists the positions of these Super Spirals. Based on those positions, let's create multiwavelength cutouts for each super spiral to see what is unique about this new class of objects.

## 1. Import some Python modules.

In [1]:
#Suppress unimportant warnings.
import warnings
warnings.filterwarnings("ignore")

import pyvo as vo
#Import the astropy Table module.
from astropy.table import Table

## 2. Search NED for objects in this paper.

Insert a Code Cell below by clicking on the "Insert" Menu and choosing "Insert Cell Below". Then consult QuickReference.md to figure out how to use astroquery to search NED for all objects in a paper, based on the refcode of the paper. Inspect the resulting astropy table.

In [2]:
from astroquery.ned import Ned
objects_in_paper = Ned.query_refcode('2016ApJ...817..109O')
objects_in_paper.show_in_notebook()

idx,No.,Object Name,RA,DEC,Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Separation,References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,,degrees,degrees,,km / s,,,,arcm,,,,,,,
0,1,WISEA J001550.14-100242.3,3.95892,-10.04511,G,52788.0,0.176083,,17.5g,--,10,0,63,7,4,10,0
1,2,WISEA J003807.80-010936.7,9.53252,-1.16024,G,62565.0,0.208695,,18.0g,--,11,0,71,10,5,10,0
2,3,WISEA J040422.92-054134.8,61.09548,-5.69303,G,75138.0,0.250635,,18.6g,--,5,0,61,10,6,8,0
3,4,WISEA J073806.16+282359.6,114.52568,28.39995,G,69255.0,0.23101,,18.0g,--,8,0,66,7,4,10,0
4,5,2MASX J07550424+1353261,118.76776,13.89065,G,66754.0,0.222669,,18.5g,--,7,0,38,6,4,6,0
5,6,WISEA J082655.11+181147.7,126.72969,18.19665,G,79608.0,0.265543,,18.4g,--,4,0,46,6,4,6,0
6,7,2MASX J08542169+0449308,133.59033,4.82511,G,47018.0,0.156835,,16.7g,--,12,0,58,7,4,10,0
7,8,WHL J090944.8+222607,137.4367,22.43539,GClstr,90837.0,0.303,PHOT,,--,5,0,0,4,5,0,0
8,9,2MASX J09094480+2226078,137.4367,22.43538,G,85557.0,0.285386,,19.0g,--,6,0,30,5,6,6,0


## 3. Filter the NED results.

The results from NED will include galaxies, but also other kinds of objects. Print the 'Type' column to see the full range of classifications. Next, print the 'Type' of just the first source in the table, in order to determine its data type (since Python 3 distinguishes between strings and byte strings). Finally, use the data type information to filter the results so that we only keep the galaxies in the list.

In [3]:
objects_in_paper['Type']

G
G
G
G
G
G
G
GClstr
G
G
GClstr


In [4]:
#Let's see whether we are dealing with strings or byte strings
objects_in_paper[0]['Type']

b'G'

In [5]:
#Byte strings it is!
galaxies = objects_in_paper[objects_in_paper['Type'] == b'G']

galaxies.show_in_notebook()

idx,No.,Object Name,RA,DEC,Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Separation,References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,,degrees,degrees,,km / s,,,,arcm,,,,,,,
0,1,WISEA J001550.14-100242.3,3.95892,-10.04511,G,52788.0,0.176083,,17.5g,--,10,0,63,7,4,10,0
1,2,WISEA J003807.80-010936.7,9.53252,-1.16024,G,62565.0,0.208695,,18.0g,--,11,0,71,10,5,10,0
2,3,WISEA J040422.92-054134.8,61.09548,-5.69303,G,75138.0,0.250635,,18.6g,--,5,0,61,10,6,8,0
3,4,WISEA J073806.16+282359.6,114.52568,28.39995,G,69255.0,0.23101,,18.0g,--,8,0,66,7,4,10,0
4,5,2MASX J07550424+1353261,118.76776,13.89065,G,66754.0,0.222669,,18.5g,--,7,0,38,6,4,6,0
5,6,WISEA J082655.11+181147.7,126.72969,18.19665,G,79608.0,0.265543,,18.4g,--,4,0,46,6,4,6,0
6,7,2MASX J08542169+0449308,133.59033,4.82511,G,47018.0,0.156835,,16.7g,--,12,0,58,7,4,10,0
7,9,2MASX J09094480+2226078,137.4367,22.43538,G,85557.0,0.285386,,19.0g,--,6,0,30,5,6,6,0
8,10,2MASX J09260805+2405242,141.53357,24.09004,G,66689.0,0.222451,,17.8g,--,12,0,32,6,7,6,0


## 4. Search the NAVO Registry for image resources.

The paper selected super spirals using WISE, SDSS, and GALEX images. Search the NAVO registry for all image resources, using the 'service_type' search parameter. How many image resources are currently available?

In [9]:
image_services = vo.regsearch(servicetype='image')
image_services.table['ivoid','short_name','res_description']

ivoid,short_name,res_description
object,object,object
ivo://3crsnapshots/sia,3CRSnap.sia,Simple Image Access Service for 3CRSnapshot resources managed at the Observatoire de Strasbourg
ivo://3crsnapshots/sia,3CRSnap.sia,Simple Image Access Service for 3CRSnapshot resources managed at the Observatoire de Strasbourg
ivo://archive.stsci.edu/borg,BoRG,"The Brightest of Reionizing Galaxies (BoRG) observations were first obtained in HST Cycle 17 in program GO 11700 (PI: Michele Trenti). BoRG is an ongoing pure-parallel program that obtains WFC3 imaging in four filters (F606W, F098M, F125W, F160W) on random sightlines at high Galactic latitudes (|b| > 30 degrees). BoRG also assimilates data from the similar Hubble Infrared Pure Parallel Imaging Extragalactic Survey (HIPPIES) pure-parallel program (GO 11702; PI: Yan). Data from the HIPPIES program uses the F600LP filter instead of F606W."
ivo://archive.stsci.edu/ghosts,GHOSTS,"The GHOSTS survey is the largest study to date of the resolved stellar populations in the outskirts of disk galaxies. The sample consists of 14 disk galaxies within 17 Mpc, whose outer disks and halos are imaged with the Hubble Space Telescope Advanced Camera for Surveys (ACS)."
ivo://archive.stsci.edu/sia/galex,GALEX,"The Galaxy Evolution Explorer (GALEX), a NASA Small Explorer mission, is performing the first all-sky, deep imaging and spectroscopic ultraviolet surveys in space. The prime goal of GALEX is to study star formation in galaxies and its evolution with time."
ivo://archive.stsci.edu/siap/hla,HLA,"The Hubble Space Telecope Legacy Archive (HLA) was developed at the Space Telescope Science Institute to optimize the science return from HST instruments. This resource is an image service which accesses all HLA observation data. The calibrated data is fully online with several forms of access including footprint visualization, composite images, extracted spectra and source lists."
ivo://arvo/siap,ArVO SIAP,"Armenian Virtual Observatory SIAP API service gives possibility to make a SIAP http request to ArVO astronomical database, which contains the data gained by Byurakan Observatory. The main part of ArVO astronomical data is the First Byurakan Survey (FBS), which is the largest and the first systematic objective prism survey of the extragalactic sky. It covers 17,000 sq.deg. in the Northern sky together with a high galactic latitudes region in the Southern sky."
ivo://cadc.nrc.ca/sia,,Image search and retrieval for CADC imaging data.
ivo://cds.vizier/siap,VizieR SIA,This service provides SIA access for VizieR associated data in FITS format.


## 5. Search the NAVO Registry for image resources that will allow you to search for Galex images.

There are hundreds of image resources...too many to quickly read through. Try adding the 'keyword' search parameter to your registry search, and find the image resource you would need to search the AllWISE images.

In [23]:
galex_image_services = vo.regsearch(keywords=['galex'], servicetype='image')
galex_image_services.table['ivoid','short_name','res_description']

ivoid,short_name,res_description
object,object,object
ivo://archive.stsci.edu/sia/galex,GALEX,"The Galaxy Evolution Explorer (GALEX), a NASA Small Explorer mission, is performing the first all-sky, deep imaging and spectroscopic ultraviolet surveys in space. The prime goal of GALEX is to study star formation in galaxies and its evolution with time."
ivo://irsa.ipac/spitzer/images/lvl,LVL,"LVL consists of a sample of 258 galaxies, which have been mapped with both IRAC (4 bands) and MIPS (3 bands). In addition, ancillary data products consisting of images in the narrow-band H-alpha line emission and broad-band R (from the ground) and the UV continuum (2 bands) from GALEX are also available for many of the galaxies."
ivo://irsa.ipac/wise/images/z0mgs,z0MGS,"z0MGS is an archival project combining WISE and GALEX images of nearby galaxies. The main sample consists of ~11,000 galaxies that are deemed to have >10% probability of being within D %lt; 50 Mpc and of having MB %lt; -18. In addition, in the course of iterating on distance estimates when creating the atlas, the z0MGS team generated images for ~5, 000 additional galaxies. These are also included in the delivery, although they do not meet the formal selection criteria. All galaxies included in the atlas have WISE W1 coverage, at minimum. In total, out of the 15,748 galaxies in DR1, 15,716 have coverage in all WISE bands, 11,687 have GALEX NUV and 10,754 have GALEX FUV. If you use z0MGS data, please cite Leroy et al. (2019)"
ivo://mast.stsci/siap/galex_atlas,GALEX_Atlas,"The GALEX Atlas of Nearby Galaxies contains images of 1034 nearby galaxies recently observed by the GALEX satellite in its far-ultraviolet (FUV; 1516A) and near-ultraviolet (NUV; 2267A) bands. The Atlas was prepared by A. Gil de Paz,S. Boissier, B.F. Madore, M. Seibert and associated members of the GALEX Team. The full paper is posted on astroph/0606440 and will be published in ApJS in 2007."
ivo://nasa.heasarc/skyview/galex,GALEX,"The GALEX, Galaxy Explorer, mission was launched by a Pegasus-XL vehicle on April 28 2003 into a 690km altitude, 29 degree inclination, circular orbit with a 98.6 minute period. The GALEX instrument allows imaging and spectroscopic observations to be made in two ultraviolet bands, Far UV (FUV) 1350-1780A and Near UV (NUV) 1770-2730A. The instrument provides simultaneous co-aligned FUV and NUV images with spatial resolution 4.3 and 5.3 arcseconds respectively. Details of the performance of the instrument and detectors can be found in Morrissey et al. (2007) ApJS, 173, 682. &lt;p&gt; The &lt;i&gt;SkyView&lt;/i&gt; GALEX surveys mosaic the intensity images of All-Sky Survey images. For a given pixel only the nearest image is used. Since a given GALEX observation is circular, this maximizes the coverage compared with default image finding algorithms which use the distance from edge of the image. &lt;p&gt; As of February 10, 2011, SkyView uses the GALEX GR6 data release."


## 6. Choose the Galex image service that you are interested in.

Note that we select the HEASARC.  The STScI also works, though it doesn't currently allow the format specification.  See the known issues.

In [42]:
galex_image_service = galex_image_services[4]
galex_image_service.describe()

Image Data Service
Galaxy Explorer All Sky Survey: Near UV
Short Name: GALEX
IVOA Identifier: ivo://nasa.heasarc/skyview/galex
Base URL: https://skyview.gsfc.nasa.gov/cgi-bin/vo/sia.pl?survey=galex&

The GALEX, Galaxy Explorer, mission was launched by a Pegasus-XL vehicle on
April 28 2003 into a 690km altitude, 29 degree inclination, circular orbit
with a 98.6 minute period. The GALEX instrument allows imaging and
spectroscopic observations to be made in two ultraviolet bands, Far UV (FUV)
1350-1780A and Near UV (NUV) 1770-2730A. The instrument provides simultaneous
co-aligned FUV and NUV images with spatial resolution 4.3 and 5.3 arcseconds
respectively. Details of the performance of the instrument and detectors can
be found in Morrissey et al. (2007) ApJS, 173, 682. <p> The <i>SkyView</i>
GALEX surveys mosaic the intensity images of All-Sky Survey images.  For a
given pixel only the nearest image is used. Since a given GALEX observation is
circular, this maximizes the coverage compar

## 7. Choose one of the galaxies in the NED list.


In [45]:
from astropy.coordinates import SkyCoord
ra = 3.95892
dec = -10.045109999999999
pos = SkyCoord(ra, dec, unit = 'deg')

## 8. Search for a list of Galex images that cover this galaxy.   How many images are returned for this one position? 

In [56]:
galex_image_table=galex_image_service.search(pos=pos,size=0.01)
galex_image_table.table

Survey,Ra,Dec,Dim,Size,Scale,Format,PixFlags,URL,LogicalName
object,float64,float64,int32,object,object,object,object,object,object
galexnear,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/fits,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexnear&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569110852&return=FITS,1
galexnear,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/tiff,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexnear&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569110852&nofits=1&quicklook=tiff&return=tiff,1
galexnear,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/gif,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexnear&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569110852&nofits=1&quicklook=gif&return=gif,1
galexnear,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/png,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexnear&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569110852&nofits=1&quicklook=png&return=png,1
galexnear,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/jpeg,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexnear&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569110852&nofits=1&quicklook=jpeg&return=jpeg,1
galexfar,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/fits,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexfar&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569111177&return=FITS,2
galexfar,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/tiff,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexfar&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569111177&nofits=1&quicklook=tiff&return=tiff,2
galexfar,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/gif,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexfar&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569111177&nofits=1&quicklook=gif&return=gif,2
galexfar,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/png,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexfar&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569111177&nofits=1&quicklook=png&return=png,2


## 9.Which format are you most interested in?

In [57]:
galex_image_table=galex_image_service.search(pos=pos,size=0.01,verbose=1,format='image/fits')
galex_image_table.table

Survey,Ra,Dec,Dim,Size,Scale,Format,PixFlags,URL,LogicalName
object,float64,float64,int32,object,object,object,object,object,object
galexnear,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/fits,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexnear&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569114445&return=FITS,1
galexfar,3.95892,-10.04511,2,[300 300],[-3.3333333333333335e-05 3.3333333333333335e-05],image/fits,F,https://skyview.gsfc.nasa.gov/cgi-bin/images?position=3.95892%2C-10.04511&survey=galexfar&pixels=300%2C300&sampler=LI&size=0.01%2C0.01&projection=Tan&coordinates=J2000.0&requestID=skv1575569114756&return=FITS,2


## 10. Filter the list of returned Galex images to select the near-band

In [55]:
galex_image_table[galex_image_table.table['Survey'] == 'galexnear']


KeyError: 'No such column: [False False]'

## 11. Visualize this AllWISE image.

In [ ]:
allwise_w1_image = Image.get_fits(allwise_w1_images[0])

In [ ]:
import matplotlib.pyplot as plt
from astropy.wcs import WCS
%matplotlib inline  

fig = plt.figure()

wcs = WCS(allwise_w1_image[0].header)
ax = fig.add_subplot(1, 1, 1, projection=wcs)
ax.imshow(allwise_w1_image[0].data, cmap='gray_r', origin='lower', vmax = 10)
ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')

## 12. Plot a cutout of the AllWISE image, centered on your position.

Try a 60 arcsecond cutout.

In [ ]:
from astropy.nddata import Cutout2D

size = 60
cutout = Cutout2D(allwise_w1_image[0].data, pos, (size, size), wcs=wcs)
wcs = cutout.wcs

fig = plt.figure()

ax = fig.add_subplot(1, 1, 1, projection=wcs)
ax.imshow(cutout.data, cmap='gray_r', origin='lower', vmax = 10)
ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')


## 13. Try visualizing a cutout of a GALEX image that covers your position.

In [ ]:
galex_image_services = Registry.query(keyword='galex', service_type='image', publisher = 'Space Telescope Science Institute')
#print(f'{len(galex_image_services)} result(s) found.')

galex_image_service = galex_image_services[0]
#galex_image_serviceservice,coords=pos,radius=0)

galex_image_table = Image.query(service=galex_image_service,coords=pos,radius=0)
galex_images = galex_image_table[0]
#galex_image_table[0]

galex_image_table[0]['contentType']
galex_fits_images = galex_images[galex_images['contentType'] == 'image/fits']
#galex_fits_images

galex_nuv_fits_images = galex_fits_images[galex_fits_images['enrValue']==2.35e-07]
#galex_nuv_fits_images

galex_nuv_fits_coadd = galex_nuv_fits_images[0]
#galex_nuv_fits_coadd

galex_nuv_image = Image.get_fits(galex_nuv_fits_coadd)

In [ ]:
import numpy as np
image_data = galex_nuv_image[0].data
print('Min:', np.min(image_data))
print('Max:', np.max(image_data))
print('Mean:', np.mean(image_data))
print('Stdev:', np.std(image_data))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=WCS(galex_nuv_image[0].header))
ax.imshow(galex_nuv_image[0].data, cmap='gray_r', origin='lower', vmin=0.0, vmax=0.01)
ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')   

In [ ]:
cutout = Cutout2D(galex_nuv_image[0].data, pos, size, wcs=WCS(galex_nuv_image[0].header))

fig = plt.figure()

ax = fig.add_subplot(1, 1, 1, projection=cutout.wcs)
ax.imshow(cutout.data, cmap='gray_r', origin='lower', vmin = 0.0, vmax = 0.01)
ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')


## 14. Try visualizing a cutout of an SDSS image that covers your position.

In [ ]:
import astropy.visualization as vis

sdss_image_services = Registry.query(keyword='SDSS', service_type='image', publisher = 'The Johns Hopkins University')
sdss_image_service = sdss_image_services[2]
#sdss_image_service

sdss_image_table = Image.query(service=sdss_image_service,coords=pos,radius=0)
sdss_images = sdss_image_table[0]
#sdss_images

#sdss_image_table[0]['Title']
sdss_rband_images = sdss_images[sdss_images['Title'] == 'Sloan Digital Sky Survey - Filter r']
#sdss_rband_images[0]

sdss_rband_image = Image.get_fits(sdss_rband_images[0])

fig = plt.figure()

ax = fig.add_subplot(1, 1, 1, projection=WCS(sdss_rband_image[0].header))

interval = vis.PercentileInterval(99.9)
vmin,vmax = interval.get_limits(sdss_rband_image[0].data)
norm = vis.ImageNormalize(vmin=vmin, vmax=vmax, stretch=vis.LogStretch(1000))
ax.imshow(sdss_rband_image[0].data, cmap = 'gray_r', norm = norm, origin = 'lower')          
ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')

In [ ]:
cutout = Cutout2D(sdss_rband_image[0].data, pos, size, wcs=WCS(sdss_rband_image[0].header))
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=cutout.wcs)
vmin,vmax = interval.get_limits(sdss_rband_image[0].data)
norm = vis.ImageNormalize(vmin=vmin, vmax=vmax, stretch=vis.LogStretch(1000))
ax.imshow(cutout.data, cmap = 'gray_r', norm = norm, origin = 'lower')          
ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')

In [ ]:
sdss_image_services

In [ ]:
#Pick the first 3 galaxies.
galaxy_subset = galaxies[0:3]

#For each galaxy,
for galaxy in galaxy_subset:

    #Establish the position.
    ra = galaxy['RA(deg)']
    dec = galaxy['DEC(deg)']
    pos = SkyCoord(ra, dec, unit = 'deg')    
    
    #Set up the plot for this position.
    fig = plt.figure(figsize=(20,6))
    plt.suptitle('POSITION = ' + str(ra) + ', ' + str(dec), fontsize=16)

    #GALEX
    
    #Find the GALEX images that overlap the position.
    galex_image_table = Image.query(service=galex_image_service,coords=pos,radius=0)
    galex_images = galex_image_table[0]
    
    #Find the GALEX All-Sky Image Survey (AIS) Near-UV FITS coadd. 
    for galex_image in galex_images:
        if (galex_image['contentType'] == 'image/fits') and galex_image['enrValue']==2.35e-07 and 'AIS' in galex_image['accessURL'] and 'main' in galex_image['accessURL']:
 
            #Create a cutout.
            gimage = Image.get_fits(galex_image)
            galex_cutout = Cutout2D(gimage[0].data, pos, size, wcs=WCS(gimage[0].header))
            
            #Plot the cutout in the first position of a 1x3 (rowsxcols) grid.
            ax = fig.add_subplot(1, 3, 1, projection=galex_cutout.wcs)
            ax.set_title(galex_image['name'])
            ax.imshow(galex_cutout.data, cmap='gray_r', origin='lower', vmin = 0.0, vmax = 0.01)
            ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')

    #SDSS
    
    #Find the SDSS images that overlap the position.
    sdss_image_table = Image.query(service=sdss_image_service,coords=pos,radius=0)
    sdss_images = sdss_image_table[0]
    
    #Find the SDSS r-band image.
    sdss_rband_images = sdss_images[sdss_images['Title'] == 'Sloan Digital Sky Survey - Filter r']
    sdss_rband_image = Image.get_fits(sdss_rband_images[0])
    
    #Create a cutout.
    sdss_rband_image = Image.get_fits(sdss_rband_images[0])
    sdss_cutout = Cutout2D(sdss_rband_image[0].data, pos, size, wcs=WCS(sdss_rband_image[0].header))
    
    #Plot the cutout in the second position of a 1x3 grid.
    vmin,vmax = interval.get_limits(sdss_cutout.data)
    norm = vis.ImageNormalize(vmin=vmin, vmax=vmax, stretch=vis.LogStretch(1000))
    ax = fig.add_subplot(1, 3, 2, projection=sdss_cutout.wcs)
    ax.imshow(sdss_cutout.data, cmap = 'gray_r', norm = norm, origin = 'lower')          
    ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')
    ax.set_title(sdss_rband_images[0]['url'])
    
    #AllWISE
    
    #Find the AllWISE images that overlap the position.
    allwise_image_table=Image.query(service=allwise_image_service,coords=pos,radius=0)
    allwise_images = allwise_image_table[0]
    
    #Find the AllWISE W1 channel images.
    allwise_w1_images = allwise_images[allwise_images['sia_bp_id'] == 'W1']
    
    #Create a cutout.
    allwise_w1_image = Image.get_fits(allwise_w1_images[0])
    allwise_cutout = Cutout2D(allwise_w1_image[0].data, pos, (size, size), wcs=WCS(allwise_w1_image[0].header))

    #Plot the cutout in the third position of a 1x3 grid.
    ax = fig.add_subplot(1, 3, 3, projection=allwise_cutout.wcs)
    ax.imshow(allwise_cutout.data, cmap='gray_r', origin='lower', vmax = 10)
    ax.scatter(ra, dec, transform=ax.get_transform('fk5'), s=500, edgecolor='red', facecolor='none')
    ax.set_title(allwise_w1_images[0]['sia_title'])
